In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression

Mounted at /content/drive


Lets include all the necessary tables from the csv files

In [ ]:
awards_players = pd.read_csv("/content/drive/Shareddrives/ML 2024/dataset/awards_players.csv")
awards_players.designation = 'awards_players'

players_teams = pd.read_csv("/content/drive/Shareddrives/ML 2024/tables/players_teams_cleaned.csv")
players_teams.designation = 'players_teams'

players = pd.read_csv("/content/drive/Shareddrives/ML 2024/dataset/players.csv")
players.designation = 'players'

teams_and_coaches = pd.read_csv("/content/drive/Shareddrives/ML 2024/tables/teams&coaches.csv")
teams_and_coaches.designation = 'teams_and_coaches'

### Dealing with player-related tables

In this notebook we will transform the players_teams table from the Player Feature Extraction Notebook into a table we can add to the main table, which is teams&coaches from the Data Exploration notebook

Let's turn awards_players table cumulative. So we will build a new table, called awards that has the total number of awards that a player has up to that year.

It is worth noting that the original awards table, for some reason, also contains the awards for the coaches, which was dealt in the Data Explration notebook.

In [ ]:
players_teams.dropna(subset=['pos'], inplace=True)

In [ ]:
awards = awards_players.groupby(['playerID', 'year'])['award'].count().reset_index()

years = range(1, 12)

players = players.rename(columns={'bioID': 'playerID'})

player_list = players['playerID'].unique()

temp = pd.MultiIndex.from_product([player_list, years], names=['playerID', 'year']).to_frame(index=False)

awards = temp.merge(awards, on=['playerID', 'year'], how='left')

awards['award'] = awards['award'].fillna(0)

awards['cumulative_awards'] = awards.groupby('playerID')['award'].cumsum()
del awards['award']

awards['cumulative_awards'] = awards['cumulative_awards'].astype(int)

awards.head()

,playerID,year,cumulative_awards
0,abrahta01w,1,0
1,abrahta01w,2,0
2,abrahta01w,3,0
3,abrahta01w,4,0
4,abrahta01w,5,0


Lets merge awards with players_teams

In [ ]:
players_teams = players_teams.merge(awards, on=['playerID', 'year'], how='left')


In [ ]:
players_teams.head()

,playerID,year,stint,tmID,rebounds,assists,blocks,dq,PostAssists,PostBlocks,...,PostftRate,PostthreeRate,PostPoints_per_minute,PostRebounds_per_minute,PostSteals_per_minute,PostTurnovers_per_minute,PostPF_per_minute,minutes_per_game,Postminutes_per_game,cumulative_awards
0,abrossv01w,2,0,MIN,174.0,53.0,9.0,2.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.538462,0.0,0
1,abrossv01w,3,0,MIN,146.0,60.0,10.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.814815,0.0,0
2,abrossv01w,4,0,MIN,141.0,82.0,11.0,0.0,4.0,1.0,...,1.0,0.428571,0.333333,0.072464,0.057971,0.115942,0.115942,26.400000,23.0,0
3,abrossv01w,5,0,MIN,74.0,45.0,2.0,0.0,3.0,2.0,...,0.5,0.250000,0.298507,0.134328,0.014925,0.044776,0.104478,21.000000,33.5,0
4,abrossv01w,6,0,MIN,107.0,60.0,6.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.064516,0.0,0


In a given year, we only know  the statistics of a player in the previous year.

So, the statistics of each player should be associated with the year before. For example, in year 5 we only know the statistics from year 4, so the table should account for that.

Also there are players that skip years so we will add the statistics from the last year a player has played.

In [ ]:
players_teams.sort_values(by=['playerID', 'year'], inplace=True)

In [ ]:
cols = players_teams.columns.difference(['playerID', 'year', 'tmID', 'franchID', 'pos', 'cumulative_awards'])

In [ ]:
def player_from_prev_year(playerID, year, dataframe=players_teams):
    for i in range(year-1, 0, -1):
      if dataframe[(dataframe['playerID'] == playerID) & (dataframe['year'] == i)].any().sum():
        return dataframe[(dataframe['playerID'] == playerID) & (dataframe['year'] == i)]
    return None

In [ ]:
players_teams_copy = players_teams.copy()

def apply_prev_year_data(row):
  prev_year = player_from_prev_year(row['playerID'], row['year'], players_teams_copy)
  if prev_year is not None:
    return prev_year[cols].values[0]
  else:
    return pd.Series(None, index=cols)

players_teams[cols] = players_teams.apply(apply_prev_year_data, axis=1, result_type='expand')

Our objective with this project is, in the year 10, try to predict what teams, in the year 11, will go to the playoffs, given the players and coach of year 11.

As we are predicting if a team will go to the playoffs in the next year, we should add the players of the following year in the current one. So we will subtract 1 to every player

In [ ]:
players_teams['year'] = players_teams['year'] - 1

In practice, with the previous steps, in a given year X, we selected the players from year X+1, while keping the statistic from year X

Now, lets count the number of new players by team by year, the players with NaN in most fields. Given the previous steps, players with 'stint' equal NaN, also have the remaining statistics equal to NaN.

In [ ]:
new_player_count = players_teams.loc[players_teams['stint'].isna()].groupby(['year', 'franchID'])['playerID'].count()

new_player_count = pd.DataFrame(new_player_count)

new_player_count.rename(columns={'playerID': 'no_new_players'}, inplace=True)

new_player_count.reset_index(inplace=True)

new_player_count.head()

,year,franchID,no_new_players
0,0,CHA,13
1,0,CLE,13
2,0,CON,13
3,0,DET,13
4,0,HOU,12


In [ ]:
new_player_count.head()

,year,franchID,no_new_players
0,0,CHA,13
1,0,CLE,13
2,0,CON,13
3,0,DET,13
4,0,HOU,12


Now lets drop all those new players

In [ ]:
players_teams = players_teams.dropna(subset=['stint'])

In [ ]:
players_teams.reset_index(drop=True, inplace=True)

In [ ]:
nan_counts = players_teams.isna().sum()

nan_counts

,0
playerID,0
year,0
stint,0
tmID,0
rebounds,0
assists,0
blocks,0
dq,0
PostAssists,0
PostBlocks,0


We have features with very different scales, each makes it difficult to create a uniform rating, so we decided to normalize the values in the players_teams table, except for 'playerID', 'year', 'tmID', 'franchID', 'pos' and 'cumulative_awards'.

In [ ]:
normalized_players_teams = normalize(players_teams[players_teams.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'cumulative_awards'})])
normalized_players_teams = pd.DataFrame(normalized_players_teams, columns=players_teams.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'cumulative_awards'}))
normalized_players_teams['playerID'] = players_teams['playerID']
normalized_players_teams['year'] = players_teams['year']
normalized_players_teams['tmID'] = players_teams['tmID']
normalized_players_teams['franchID'] = players_teams['franchID']
normalized_players_teams['pos'] = players_teams['pos']
normalized_players_teams['cumulative_awards'] = players_teams['cumulative_awards']

In [ ]:
normalized_players_teams

,PF_per_minute,PostAssists,PostBlocks,PostDQ,PostPF_per_minute,PostPoints_per_minute,PostRebounds_per_minute,PostSteals_per_minute,PostTurnovers_per_minute,PostfgRate,...,steals_per_minute,stint,threeRate,turnovers_per_minute,playerID,year,tmID,franchID,pos,cumulative_awards
0,0.000415,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000249,0.000000,0.001255,0.000504,abrossv01w,2,MIN,MIN,F,0
1,0.000512,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000295,0.000000,0.001882,0.000645,abrossv01w,3,MIN,MIN,F,0
2,0.000545,0.021872,0.005468,0.0,0.000634,0.001823,0.000396,0.000317,0.000634,0.001491,...,0.000304,0.000000,0.001667,0.000621,abrossv01w,4,MIN,MIN,F,0
3,0.000753,0.024863,0.016576,0.0,0.000866,0.002474,0.001113,0.000124,0.000371,0.002883,...,0.000538,0.000000,0.003127,0.000771,abrossv01w,5,MIN,MIN,F,0
4,0.000760,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000424,0.000000,0.002765,0.000707,abrossv01w,6,MIN,MIN,F,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409,0.000791,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000245,0.000000,0.001975,0.000481,youngta01w,9,CHI,CHI,F,0
1410,0.001627,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001084,0.013195,0.002639,0.001627,youngta01w,10,CHI,CHI,F,0
1411,0.001892,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000210,0.000000,0.004648,0.000799,zakalok01w,2,DET,DET,C,0
1412,0.001892,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000210,0.000000,0.004648,0.000799,zakalok01w,2,PHO,PHO,C,0


Lets add the percentage of won games to player. This corresponds to rate of games won by the teams the player was part of, while she was there.

To do this we got the 'won' column from teams_and_coaches from the team the player was part of in that year, which corresponds to the rate of won games of a team.

Then we transformed 'won' to the average of all previous years, for each player.

In [ ]:
normalized_players_teams = normalized_players_teams.merge(teams_and_coaches[['year', 'franchID', 'won']], on=['year', 'franchID'], how='left')

normalized_players_teams['player_year'] = 1

normalized_players_teams.head()

,PF_per_minute,PostAssists,PostBlocks,PostDQ,PostPF_per_minute,PostPoints_per_minute,PostRebounds_per_minute,PostSteals_per_minute,PostTurnovers_per_minute,PostfgRate,...,threeRate,turnovers_per_minute,playerID,year,tmID,franchID,pos,cumulative_awards,won,player_year
0,0.000415,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001255,0.000504,abrossv01w,2,MIN,MIN,F,0,0.375000,1
1,0.000512,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001882,0.000645,abrossv01w,3,MIN,MIN,F,0,0.312500,1
2,0.000545,0.021872,0.005468,0.0,0.000634,0.001823,0.000396,0.000317,0.000634,0.001491,...,0.001667,0.000621,abrossv01w,4,MIN,MIN,F,0,0.529412,1
3,0.000753,0.024863,0.016576,0.0,0.000866,0.002474,0.001113,0.000124,0.000371,0.002883,...,0.003127,0.000771,abrossv01w,5,MIN,MIN,F,0,0.529412,1
4,0.000760,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002765,0.000707,abrossv01w,6,MIN,MIN,F,0,0.411765,1


In [ ]:
#Make won cumulative, remember that it is a rating, so it must always be from 0-1
normalized_players_teams['won'] = normalized_players_teams.groupby('playerID')['won'].cumsum()
normalized_players_teams['player_year'] = normalized_players_teams.groupby('playerID')['player_year'].cumsum()

normalized_players_teams['won'] = normalized_players_teams['won'] / normalized_players_teams['player_year']

In [ ]:
del normalized_players_teams['player_year']

As the won rate only applies to games after the playoff, lets set all NaN in 'won' column to 0

In [ ]:
normalized_players_teams.loc[normalized_players_teams['won'].isna(), 'won'] = 0

It is not feaseble to have all the characteristics of all players in the main table used for training, so we will select only the top 3 players of each position.

But with this a new problem arises. How can we know what are the best players?

To solve this, we created a rating column, which has a rating for each player

A player can play in one or more of 3 positions, Foward(F), Guard(G) and Center(C)

For each position, a linear regression model is created, to try to learn which player's features are more important, based on the rate of won games.

The result of this model becomes the rating.

In [ ]:
playersF = normalized_players_teams[(normalized_players_teams['pos'] == 'F') | (normalized_players_teams['pos'].str.contains('F'))]
playersG = normalized_players_teams[(normalized_players_teams['pos'] == 'G') | (normalized_players_teams['pos'].str.contains('G'))]
playersC = normalized_players_teams[(normalized_players_teams['pos'] == 'C') | (normalized_players_teams['pos'].str.contains('C'))]

In [ ]:
X_F = playersF[playersF.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year'})]
y_F = playersF['won']

model_F = LinearRegression()
model_F.fit(X_F, y_F)

# You can also print the coefficients and intercept of the model
print(f'Coefficients: {model_F.coef_}')
print(f'Intercept: {model_F.intercept_}')

Coefficients: [ 8.85812766e+00  6.97313559e-02  6.50012220e-01 -1.37804004e+00
 -1.39146313e+01  6.57326915e+00  1.48835755e+01  6.78761885e+00
  2.62551440e+01 -3.13826261e+00  2.21868182e+00  3.24015220e-01
 -2.39190728e+00 -1.19857959e-01 -8.90050031e-02  1.38451247e-02
  9.65492982e-01  9.26016002e+00 -6.25355264e-01 -1.81401064e-01
 -6.84550026e-03 -8.80837548e+00 -1.06229607e-01 -1.32543617e+01
 -3.84014201e+00  2.96682801e+00 -1.57613328e+01]
Intercept: 0.6399949141990442


In [ ]:
X_G = playersG[playersG.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year'})]
y_G = playersG['won']

model_G = LinearRegression()
model_G.fit(X_G, y_G)

# You can also print the coefficients and intercept of the model
print(f'Coefficients: {model_G.coef_}')
print(f'Intercept: {model_G.intercept_}')

Coefficients: [-2.81323287e+01  3.94409251e-01  9.37672808e-01  7.67514717e-01
  1.88021218e+01 -7.85381390e+00  9.07371030e+00 -9.19815205e-01
  7.38928868e+00  7.81052745e+00  2.05476455e+00  4.15959506e-02
 -9.19478041e-01 -1.89518840e-01 -1.21426978e-01  5.83613284e-03
  2.27055971e+00  2.15339201e+01  9.99716058e+00 -3.55617801e-01
 -1.18104157e-01 -6.00529183e+00 -1.28734779e-01  9.89034093e+00
 -2.52884907e+00 -5.64627388e+00 -1.01346820e+01]
Intercept: 0.7629482642664311


In [ ]:
X_C = playersC[playersC.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year'})]
y_C = playersC['won']

model_C = LinearRegression()
model_C.fit(X_C, y_C)

# You can also print the coefficients and intercept of the model
print(f'Coefficients: {model_C.coef_}')
print(f'Intercept: {model_C.intercept_}')

Coefficients: [ 7.48389687e+00 -8.16048708e-01  1.14007514e+00 -2.69596360e+00
  5.68734525e+00  4.61582334e+00  8.62796044e+00  1.17594859e+01
 -7.16440565e+00  6.31351943e-01 -1.59066901e+00  4.15501239e-01
 -5.58418704e+00  1.21172736e-01 -4.92036268e-02  2.38422895e-02
 -9.83970468e-01 -7.44407450e+00  1.24720239e+00 -1.26102513e-01
 -3.23373749e-01 -3.17780132e+00 -1.25786304e-01 -9.57319051e+00
  4.34814470e+00 -3.68301348e-01 -2.66760099e+01]
Intercept: 0.671661535703743


When a player plays in more that one position, the positions she plays are delimited by "-", for example "F-G"

In [ ]:
mask = ~(normalized_players_teams['pos'].str.contains('-'))
normalized_players_teams = normalized_players_teams[mask]

def split_positions(row):
  if '-' in row['pos']:
    positions = row['pos'].split('-')
    new_rows = [row.copy().assign(pos=pos) for pos in positions]
    return pd.concat([pd.DataFrame([new_r]) for new_r in new_rows])
  else:
    return None

new_rows = normalized_players_teams.apply(split_positions, axis=1, result_type='expand').dropna()
normalized_players_teams = pd.concat([normalized_players_teams, new_rows], ignore_index=True)
normalized_players_teams.drop(columns=[0], inplace=True)

In [ ]:
print(normalized_players_teams[(normalized_players_teams['pos']!='F') & (normalized_players_teams['pos']!='G') & (normalized_players_teams['pos']!='C')])

Empty DataFrame
Columns: [PF_per_minute, PostAssists, PostBlocks, PostDQ, PostPF_per_minute, PostPoints_per_minute, PostRebounds_per_minute, PostSteals_per_minute, PostTurnovers_per_minute, PostfgRate, PostftRate, Postminutes_per_game, PostthreeRate, assists, blocks, dq, fgRate, ftRate, height, minutes_per_game, points_per_minute, rebounds, steals_per_minute, stint, threeRate, turnovers_per_minute, playerID, year, tmID, franchID, pos, cumulative_awards, won]
Index: []

[0 rows x 33 columns]


In [ ]:
normalized_players_teams.head()

,PF_per_minute,PostAssists,PostBlocks,PostDQ,PostPF_per_minute,PostPoints_per_minute,PostRebounds_per_minute,PostSteals_per_minute,PostTurnovers_per_minute,PostfgRate,...,stint,threeRate,turnovers_per_minute,playerID,year,tmID,franchID,pos,cumulative_awards,won
0,0.000415,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.001255,0.000504,abrossv01w,2.0,MIN,MIN,F,0.0,0.375000
1,0.000512,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.001882,0.000645,abrossv01w,3.0,MIN,MIN,F,0.0,0.343750
2,0.000545,0.021872,0.005468,0.0,0.000634,0.001823,0.000396,0.000317,0.000634,0.001491,...,0.0,0.001667,0.000621,abrossv01w,4.0,MIN,MIN,F,0.0,0.405637
3,0.000753,0.024863,0.016576,0.0,0.000866,0.002474,0.001113,0.000124,0.000371,0.002883,...,0.0,0.003127,0.000771,abrossv01w,5.0,MIN,MIN,F,0.0,0.436581
4,0.000760,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.002765,0.000707,abrossv01w,6.0,MIN,MIN,F,0.0,0.431618


In [ ]:
def get_rating(row, pos):
  if(pos == 'F'):
    value_for_model = row[normalized_players_teams.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year', 'rating'})]
    rating = model_F.predict(value_for_model.values.reshape(1, -1))[0]
    return rating
  elif(pos == 'G'):
    value_for_model = row[normalized_players_teams.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year', 'rating'})]
    return model_G.predict(value_for_model.values.reshape(1, -1))[0]
  elif(pos == 'C'):
    value_for_model = row[normalized_players_teams.columns.difference({'playerID', 'year', 'tmID', 'franchID', 'pos', 'won', 'player_year', 'rating'})]
    return model_C.predict(value_for_model.values.reshape(1, -1))[0]
  else:
    return 0

for index, row in normalized_players_teams.iterrows():
  normalized_players_teams.loc[index, 'rating'] = get_rating(row, row['pos'])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature n

For each team, lets select the top 3 players. First the player with max rating, then the second max and third max.

In [ ]:
def get_max_rating_player(group):
    max_rating_index = group['rating'].idxmax()
    return group.loc[max_rating_index]

max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_max_rating_player)

max_rating_players.reset_index(drop=True, inplace=True)

max_rating_players.sort_values(by=['year', 'franchID'], inplace=True)

max_columns_names = [name + '_max' if name not in ['year', 'franchID', 'pos'] else name for name in max_rating_players.columns]

max_rating_players.columns = max_columns_names

max_rating_players.head()

<ipython-input-31-df999d522106>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_max_rating_player)


,PF_per_minute_max,PostAssists_max,PostBlocks_max,PostDQ_max,PostPF_per_minute_max,PostPoints_per_minute_max,PostRebounds_per_minute_max,PostSteals_per_minute_max,PostTurnovers_per_minute_max,PostfgRate_max,...,threeRate_max,turnovers_per_minute_max,playerID_max,year,tmID_max,franchID,pos,cumulative_awards_max,won_max,rating_max
9,0.002439,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000610,erbsu01w,1.0,CHA,CHA,C,0.0,0.25000,0.439865
10,0.000856,0.025624,0.008541,0.0,0.002135,0.002912,0.001747,0.000388,0.000971,0.002669,...,0.002211,0.000637,feastal01w,1.0,CHA,CHA,F,0.0,0.25000,0.493997
11,0.000685,0.039629,0.000000,0.0,0.001039,0.002209,0.000780,0.000520,0.000260,0.001981,...,0.002433,0.000539,edwarto01w,1.0,CHA,CHA,G,0.0,0.25000,0.535606
41,0.001943,0.000000,0.000000,0.0,0.006414,0.000000,0.006414,0.000000,0.000000,0.000000,...,0.000000,0.001166,sauerpa01w,1.0,CLE,CLE,C,0.0,0.53125,0.532204
42,0.001559,0.028057,0.000000,0.0,0.000000,0.004676,0.002338,0.000000,0.002338,0.007014,...,0.000000,0.001559,barnead01w,1.0,CLE,CLE,F,0.0,0.53125,0.570828


In [ ]:
def get_second_max_rating_player(group):
    # Drop the row with the max rating
    new_group = group.drop(group['rating'].idxmax())
    # Find the second max rating index
    if len(new_group) > 0:
      second_max_rating_index = group['rating'].idxmax()
      return group.loc[second_max_rating_index]
    else:
      return group.loc[group['rating'].idxmax()]

second_max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_second_max_rating_player)

second_max_rating_players.reset_index(drop=True, inplace=True)

second_max_rating_players.sort_values(by=['year', 'franchID'], inplace=True)

second_max_columns_names = [name + '_second_max' if name not in ['year', 'franchID', 'pos'] else name for name in second_max_rating_players.columns]

second_max_rating_players.columns = second_max_columns_names

second_max_rating_players.head()


<ipython-input-32-6c129236d257>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  second_max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_second_max_rating_player)


,PF_per_minute_second_max,PostAssists_second_max,PostBlocks_second_max,PostDQ_second_max,PostPF_per_minute_second_max,PostPoints_per_minute_second_max,PostRebounds_per_minute_second_max,PostSteals_per_minute_second_max,PostTurnovers_per_minute_second_max,PostfgRate_second_max,...,threeRate_second_max,turnovers_per_minute_second_max,playerID_second_max,year,tmID_second_max,franchID,pos,cumulative_awards_second_max,won_second_max,rating_second_max
9,0.002439,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000610,erbsu01w,1.0,CHA,CHA,C,0.0,0.25000,0.439865
10,0.000856,0.025624,0.008541,0.0,0.002135,0.002912,0.001747,0.000388,0.000971,0.002669,...,0.002211,0.000637,feastal01w,1.0,CHA,CHA,F,0.0,0.25000,0.493997
11,0.000685,0.039629,0.000000,0.0,0.001039,0.002209,0.000780,0.000520,0.000260,0.001981,...,0.002433,0.000539,edwarto01w,1.0,CHA,CHA,G,0.0,0.25000,0.535606
41,0.001943,0.000000,0.000000,0.0,0.006414,0.000000,0.006414,0.000000,0.000000,0.000000,...,0.000000,0.001166,sauerpa01w,1.0,CLE,CLE,C,0.0,0.53125,0.532204
42,0.001559,0.028057,0.000000,0.0,0.000000,0.004676,0.002338,0.000000,0.002338,0.007014,...,0.000000,0.001559,barnead01w,1.0,CLE,CLE,F,0.0,0.53125,0.570828


In [ ]:
def get_third_max_rating_player(group):
    # Drop the rows with the max and second max ratings
    new_group = group.drop(group['rating'].idxmax())
    if len(new_group) == 0:
      return group.loc[group['rating'].idxmax()]

    new_group_2 = new_group.drop(new_group['rating'].idxmax())
    if len(new_group_2) == 0:
      return new_group.loc[new_group['rating'].idxmax()]
    # Find the third max rating index
    third_max_rating_index = group['rating'].idxmax()
    return group.loc[third_max_rating_index]

third_max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_third_max_rating_player)

third_max_rating_players.reset_index(drop=True, inplace=True)

third_max_rating_players.sort_values(by=['year', 'franchID'], inplace=True)

third_max_columns_names = [name + '_third_max' if name not in ['year', 'franchID', 'pos'] else name for name in third_max_rating_players.columns]

third_max_rating_players.columns = third_max_columns_names

third_max_rating_players.head()

<ipython-input-33-3d758359907b>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  third_max_rating_players = normalized_players_teams.groupby(['franchID', 'year', 'pos']).apply(get_third_max_rating_player)


,PF_per_minute_third_max,PostAssists_third_max,PostBlocks_third_max,PostDQ_third_max,PostPF_per_minute_third_max,PostPoints_per_minute_third_max,PostRebounds_per_minute_third_max,PostSteals_per_minute_third_max,PostTurnovers_per_minute_third_max,PostfgRate_third_max,...,threeRate_third_max,turnovers_per_minute_third_max,playerID_third_max,year,tmID_third_max,franchID,pos,cumulative_awards_third_max,won_third_max,rating_third_max
9,0.002439,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000610,erbsu01w,1.0,CHA,CHA,C,0.0,0.25000,0.439865
10,0.000946,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.002236,0.000516,smithch02w,1.0,CHA,CHA,F,0.0,0.25000,0.435059
11,0.000685,0.039629,0.0,0.0,0.001039,0.002209,0.000780,0.00052,0.000260,0.001981,...,0.002433,0.000539,edwarto01w,1.0,CHA,CHA,G,0.0,0.25000,0.535606
41,0.001943,0.000000,0.0,0.0,0.006414,0.000000,0.006414,0.00000,0.000000,0.000000,...,0.000000,0.001166,sauerpa01w,1.0,CLE,CLE,C,0.0,0.53125,0.532204
42,0.001559,0.028057,0.0,0.0,0.000000,0.004676,0.002338,0.00000,0.002338,0.007014,...,0.000000,0.001559,barnead01w,1.0,CLE,CLE,F,0.0,0.53125,0.570828


Now lets also add the mean rating by team by year

In [ ]:
ratings_table = normalized_players_teams[['playerID', 'year', 'franchID', 'pos', 'rating']]

ratings_table_grouped_by_pos = ratings_table.groupby(['year', 'franchID', 'pos'])['rating'].mean().reset_index()

ratings_table_grouped_by_pos.rename(columns={'rating': 'mean'}, inplace=True)

ratings_table_grouped_by_pos.head()

,year,franchID,pos,mean
0,1.0,CHA,C,0.439865
1,1.0,CHA,F,0.464528
2,1.0,CHA,G,0.495861
3,1.0,CLE,C,0.522991
4,1.0,CLE,F,0.513338


Lets now join the contents of the previous steps, grouped by 'pos', 'year' and 'franchID'.

Lets also add a new column with the number of new players by year by team

New columns: top1_F, top2_F, tp3_F, mean_F,  top1_C, top2_C, tp3_C, mean_C, top1_G, top2_G, top3_G, mean_G and no_new_players

In [ ]:
max_columns_names.remove('year')
max_columns_names.remove('franchID')
max_columns_names.remove('pos')

second_max_columns_names.remove('year')
second_max_columns_names.remove('franchID')
second_max_columns_names.remove('pos')

third_max_columns_names.remove('year')
third_max_columns_names.remove('franchID')
third_max_columns_names.remove('pos')

In [ ]:
# Create a pivot table to get top 3 ratings and mean for each position by year and franchID
top_ratings_table = pd.pivot_table(max_rating_players, values=max_columns_names, index=['year', 'franchID'], columns=['pos'], aggfunc='max')
top2_ratings_table = pd.pivot_table(second_max_rating_players, values=second_max_columns_names, index=['year', 'franchID'], columns=['pos'], aggfunc='max')
top3_ratings_table = pd.pivot_table(third_max_rating_players, values=third_max_columns_names, index=['year', 'franchID'], columns=['pos'], aggfunc='max')
mean_ratings_table = pd.pivot_table(ratings_table_grouped_by_pos, values='mean', index=['year', 'franchID'], columns=['pos'], aggfunc='mean')

# Rename columns for clarity
top_ratings_table = top_ratings_table.rename(columns={'F': 'top1_F', 'C': 'top1_C', 'G': 'top1_G'})
top2_ratings_table = top2_ratings_table.rename(columns={'F': 'top2_F', 'C': 'top2_C', 'G': 'top2_G'})
top3_ratings_table = top3_ratings_table.rename(columns={'F': 'top3_F', 'C': 'top3_C', 'G': 'top3_G'})
mean_ratings_table = mean_ratings_table.rename(columns={'F': 'mean_F', 'C': 'mean_C', 'G': 'mean_G'})

# Combine top 3 ratings and mean into a single table
final_table = pd.concat([top_ratings_table, top2_ratings_table, top3_ratings_table, mean_ratings_table], axis=1)

final_table = final_table.merge(new_player_count, on=['year', 'franchID'], how='left')

# Reset the index to have 'year' and 'franchID' as columns
final_table = final_table.reset_index()

final_table.columns.name = None

In [ ]:
final_table.head()

,index,year,franchID,"(PF_per_minute_max, top1_C)","(PF_per_minute_max, top1_F)","(PF_per_minute_max, top1_G)","(PostAssists_max, top1_C)","(PostAssists_max, top1_F)","(PostAssists_max, top1_G)","(PostBlocks_max, top1_C)",...,"(turnovers_per_minute_third_max, top3_C)","(turnovers_per_minute_third_max, top3_F)","(turnovers_per_minute_third_max, top3_G)","(won_third_max, top3_C)","(won_third_max, top3_F)","(won_third_max, top3_G)",mean_C,mean_F,mean_G,no_new_players
0,0,1.0,CHA,0.002439,0.000856,0.000685,0.000000,0.025624,0.039629,0.000000,...,0.000610,0.000516,0.000539,0.25000,0.25000,0.25000,0.439865,0.464528,0.495861,3.0
1,1,1.0,CLE,0.001943,0.001559,0.000522,0.000000,0.028057,0.063246,0.000000,...,0.001166,0.001559,0.000364,0.53125,0.53125,0.53125,0.522991,0.513338,0.509830,2.0
2,2,1.0,CON,0.000806,NaN,0.000301,0.017869,NaN,0.057090,0.041695,...,0.000392,NaN,0.000902,0.50000,NaN,0.50000,0.551317,NaN,0.489982,4.0
3,3,1.0,DET,NaN,0.000518,0.000477,NaN,0.000000,0.000000,NaN,...,NaN,0.000494,0.000874,NaN,0.43750,0.43750,NaN,0.440673,0.448281,5.0
4,4,1.0,HOU,0.000738,0.000303,0.001070,0.005898,0.037471,0.079780,0.011796,...,0.000146,0.000290,0.000343,0.84375,0.84375,0.84375,0.516522,0.555364,0.603464,4.0


In [ ]:
final_table.fillna(0, inplace=True)

In [ ]:
final_table.drop(columns=[('playerID_max', 'top1_F'),
                            ('playerID_max', 'top1_C'),
                            ('playerID_max', 'top1_G'),
                            ('playerID_second_max', 'top2_F'),
                            ('playerID_second_max', 'top2_C'),
                            ('playerID_second_max', 'top2_G'),
                            ('playerID_third_max', 'top3_F'),
                            ('playerID_third_max', 'top3_C'),
                            ('playerID_third_max', 'top3_G'),
                            ('tmID_max', 'top1_G'),
                            ('tmID_second_max', 'top2_G'),
                            ('tmID_third_max', 'top3_G'),
                            ('tmID_max', 'top1_C'),
                            ('tmID_second_max', 'top2_C'),
                            ('tmID_third_max', 'top3_C'),
                            ('tmID_max', 'top1_F'),
                            ('tmID_second_max', 'top2_F'),
                            ('tmID_third_max', 'top3_F')
                            ], inplace=True)

In [ ]:
final_table = final_table.reset_index(drop=True)

Lets see how many columns there are in the final_table

In [ ]:
len(final_table.columns)

268

Even after all of this the table has 268 columns, which is to much, so lets use PCA to reduce the number of columns, but keeping some of the new columns.

In the end we will have 5 columns resulting from PCA, the rating of the top 3 players of each position and the mean rating of each position

In [ ]:
do_dim_reduction = True

In [ ]:
if(do_dim_reduction):
  final_table.columns = final_table.columns.astype(str)

In [ ]:
if(do_dim_reduction):
  from sklearn.decomposition import PCA

  columns_to_subtract = ['year',
                         'tmID',
                         'franchID',
                         "('rating_max', 'top1_C')",
                         "('rating_max', 'top1_F')",
                         "('rating_max', 'top1_G')",
                         "('rating_second_max', 'top2_C')",
                         "('rating_second_max', 'top2_F')",
                         "('rating_second_max', 'top2_G')",
                         "('rating_third_max', 'top3_C')",
                         "('rating_third_max', 'top3_F')",
                         "('rating_third_max', 'top3_G')",
                         'mean_C',
                         'mean_F',
                         'mean_G',
                         'no_new_players']

  features_to_reduce = final_table.columns.difference(columns_to_subtract)
  X = final_table[features_to_reduce]

  pca = PCA(n_components=5)

  X_reduced = pca.fit_transform(X)

  final_table_reduced = pd.DataFrame(data=X_reduced, columns=[f'AbstractPlayersFeature{i+1}' for i in range(5)])
  final_table_reduced['year'] = final_table['year']
  final_table_reduced['franchID'] = final_table['franchID']

  print(final_table_reduced.head())

   AbstractPlayersFeature1  AbstractPlayersFeature2  AbstractPlayersFeature3  \
0               -68.505848                -0.643121                 0.039782   
1               -67.509710                -0.660443                -0.097542   
2               -66.507513                -0.576851                 0.040465   
3               -65.500845                -0.705409                -0.081360   
4               -64.488797                -0.479669                 1.861932   

   AbstractPlayersFeature4  AbstractPlayersFeature5  year franchID  
0                -0.160352                -0.129820   1.0      CHA  
1                -0.142234                 0.025497   1.0      CLE  
2                -0.117238                -0.184575   1.0      CON  
3                -0.184703                 0.092119   1.0      DET  
4                 0.297550                 0.120350   1.0      HOU  


In [ ]:
if(do_dim_reduction):
  final_table_reduced['rating_max_C'] =  final_table["('rating_max', 'top1_C')"]
  final_table_reduced['rating_max_F'] =  final_table["('rating_max', 'top1_F')"]
  final_table_reduced['rating_max_G'] =  final_table["('rating_max', 'top1_G')"]

  final_table_reduced['rating_second_max_C'] =  final_table["('rating_second_max', 'top2_C')"]
  final_table_reduced['rating_second_max_F'] =  final_table["('rating_second_max', 'top2_F')"]
  final_table_reduced['rating_second_max_G'] =  final_table["('rating_second_max', 'top2_G')"]

  final_table_reduced['rating_third_max_C'] =  final_table["('rating_third_max', 'top3_C')"]
  final_table_reduced['rating_third_max_F'] =  final_table["('rating_third_max', 'top3_F')"]
  final_table_reduced['rating_third_max_G'] =  final_table["('rating_third_max', 'top3_G')"]

  final_table_reduced['mean_C'] =  final_table["mean_C"]
  final_table_reduced['mean_F'] =  final_table["mean_F"]
  final_table_reduced['mean_G'] =  final_table["mean_G"]

  final_table_reduced['no_new_players'] =  final_table["no_new_players"]

  final_table_reduced.dropna(subset=['no_new_players'], inplace=True)

In [ ]:
if(do_dim_reduction):
  print(final_table_reduced.head())

   AbstractPlayersFeature1  AbstractPlayersFeature2  AbstractPlayersFeature3  \
0               -68.505848                -0.643121                 0.039782   
1               -67.509710                -0.660443                -0.097542   
2               -66.507513                -0.576851                 0.040465   
3               -65.500845                -0.705409                -0.081360   
4               -64.488797                -0.479669                 1.861932   

   AbstractPlayersFeature4  AbstractPlayersFeature5  year franchID  \
0                -0.160352                -0.129820   1.0      CHA   
1                -0.142234                 0.025497   1.0      CLE   
2                -0.117238                -0.184575   1.0      CON   
3                -0.184703                 0.092119   1.0      DET   
4                 0.297550                 0.120350   1.0      HOU   

   rating_max_C  rating_max_F  rating_max_G  rating_second_max_C  \
0      0.439865      0.493997 

Lets now save this new tables into csv files

In [ ]:
final_table.to_csv("/content/drive/Shareddrives/ML 2024/tables/ratings_dataset.csv")

In [ ]:
if(do_dim_reduction):
  final_table_reduced.to_csv("/content/drive/Shareddrives/ML 2024/tables/ratings_dataset_reduced.csv")

In [ ]:
if(do_dim_reduction):
  final_table_reduced = final_table_reduced.reset_index(drop=True)

Lets merge this table with the the teams and coaches table from the Data Exploration notebook

In [ ]:
teams_and_coaches = pd.read_csv("/content/drive/Shareddrives/ML 2024/tables/teams&coaches.csv")

In [ ]:
if(do_dim_reduction):
  teams_and_coaches = teams_and_coaches.merge(final_table_reduced, on=['year', 'franchID'], how='left')
else:
  teams_and_coaches = teams_and_coaches.merge(final_table, on=['year', 'franchID'], how='left')

In [ ]:
teams_and_coaches = teams_and_coaches.drop(columns=['Unnamed: 0'])

In [ ]:
teams_and_coaches.head()

,year,tmID,franchID,confID,rank,firstRound,semis,finals,o_fga,o_fta,...,rating_second_max_C,rating_second_max_F,rating_second_max_G,rating_third_max_C,rating_third_max_F,rating_third_max_G,mean_C,mean_F,mean_G,no_new_players
0,1,LAS,LAS,0,1.0,1,0,0,1956.0,693.0,...,0.591842,0.615393,0.560788,0.472860,0.536651,0.560788,0.532351,0.576022,0.535423,2.0
1,1,NYL,NYL,1,1.0,1,1,0,1815.0,567.0,...,0.000000,0.579372,0.593629,0.000000,0.543367,0.593629,0.000000,0.561369,0.512405,6.0
2,1,CLE,CLE,1,2.0,1,0,0,1828.0,570.0,...,0.532204,0.570828,0.572901,0.532204,0.570828,0.572901,0.522991,0.513338,0.509830,2.0
3,1,HOU,HOU,0,2.0,1,1,1,1894.0,634.0,...,0.516522,0.555364,0.613733,0.516522,0.555364,0.593196,0.516522,0.555364,0.603464,4.0
4,1,ORL,CON,1,3.0,0,0,0,1911.0,546.0,...,0.551317,0.000000,0.528959,0.551317,0.000000,0.451005,0.551317,0.000000,0.489982,4.0


Lets drop rows where year is 11

In [ ]:
teams_and_coaches = teams_and_coaches[teams_and_coaches['year'] != 11]

In [ ]:
teams_and_coaches.dropna(subset=['no_new_players'], inplace=True)

In [ ]:
teams_and_coaches.to_csv("/content/drive/Shareddrives/ML 2024/tables/dataset.csv", index=False)